## Digit Recognizer de Kaggle con PCA y SVC (Kaggle_score: 0.98200) 

##### El siguiente notebook tiene como objetivo abordar el set  MNIST  propuesto en la plataforma Kaggle, utilizando support machines, principal component analysis, junto a un adecuado escalado de los datos, así como un ajuste de hiperparámetros.

##### Librerías necesarias y definiciones iniciales:

In [2]:
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, KFold
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split,cross_validate,GridSearchCV
from time import time
from scipy import stats

filename = 'D:\\Python\\Git_seguimiento\\Digital Recognizer\\train.csv'
test_filename = 'D:\\Python\\Git_seguimiento\\Digital Recognizer\\test.csv'

train_df = pd.read_csv(filename)
test_df = pd.read_csv(test_filename)

images = train_df.iloc[:,1:]
labels = train_df.iloc[:,:1]

In [3]:
images.describe()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
count,42000.0,42000.0,42000.0,42000.0,42000.0,42000.0,42000.0,42000.0,42000.0,42000.0,...,42000.000000,42000.000000,42000.000000,42000.00000,42000.000000,42000.000000,42000.0,42000.0,42000.0,42000.0
mean,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.219286,0.117095,0.059024,0.02019,0.017238,0.002857,0.0,0.0,0.0,0.0
std,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,6.312890,4.633819,3.274488,1.75987,1.894498,0.414264,0.0,0.0,0.0,0.0
min,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0
25%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0
50%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0
75%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0
max,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,254.000000,254.000000,253.000000,253.00000,254.000000,62.000000,0.0,0.0,0.0,0.0


In [11]:
train_df.shape

(42000, 785)

##### De la vista anterior se puede observar que los vectores Xi tienen distintos rangos a pesar que los pixeles pueden tener valores de 0 a 255, además hay vectores sólo de ceros. 

##### Se tienen 784 vectores Xi, correspondiente a 28x28 pixeles por imagen. 

##### Dado que se va a utilizar SVC, es importante que las variables predictivas se encuentren en un mismo rango. El escalado de los vectores Xi debe ser realizado de modo tal que se conserve la estructura general de la matriz. Por lo tanto, utilizar MaxAbsScaler, MinMaxScaler o StandarScaler no son buenas estrategias. 

##### Se optará por dividir por el mayor valor posible entre todos los rangos, el cual es 255. De este modo se obtendrá un rango para cada columna de  [0,1] conservando lo mejor posible las proporciones de los datos:

In [13]:
X = images.values/255.0
y = labels['label'].values

#### Modelo: se utilizará Pipelines de modo de encadenar un PCA con SVC, en primer lugar se realizará un ajuste del porcentaje de varianza explicada mediante Grid Search. 

##### Cabe mencionar que utilizar un total de varianza explicada elevado (mayor cantidad de vectores Xi) implica overfitting y una baja implica underfitting. PCA tiene un parámetro "n_components", que si se escoge un número entero n, se obtendrán n componentes principales. Por otra parte, si se escoge un valor decimal entre 0 y 1 PCA toma el parámetro como el % de var explicada que se desea y a posteriori determinará el número de componentes necesario para llegar a dicho porcentaje.

##### Antes de hacer el ajuste hiperparamétrico es necesario reducir la cantidad de observaciones a considerar debido al tiempo que requeriría el realizarlo con toda la base. Por lo tanto, se utilizarán 5000 observaciones solamente para reducir el tiempo de cómputo a unos pocos segundos en vez de varios minutos. 

##### Dado que la naturaleza de los datos no varía, no es necesario utilizar las 42.000 observaciones y la aproximación con 5.000 será bastante buena. 

##### Antes de ajustar el modelo final, se debe volver a setear images y labels para que contengan las 42.000 observaciones.

In [ ]:
#images = train_df.iloc[0:5000,1:]
#labels = train_df.iloc[0:5000,:1]

#Partición del set de entrenamiento para realizar ajuste hiperparamétrico

#X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, random_state=7)

In [7]:
##Tuning PCA (Valores optimos: n_components = 0.72,whiten=False)
#if __name__ == '__main__':
#    svc = SVC()
#    pca =PCA()
#    steps = [('PCA', pca),('SVC', svc)]
#    pipeline = Pipeline(steps)
#    param_grid = { "PCA__n_components" : [0.70,0.71,0.72,0.73,0.74,0.75,0.76,0.77,0.78], "PCA__whiten" :[True,False]}
#    grid = GridSearchCV(pipeline, param_grid,cv=3,scoring = "accuracy",iid=False, n_jobs=-1)
#    start = time()
#    grid.fit(X_train, Y_train)
#    print("Tuned Model Parameters: {}".format(grid.best_params_))
#    print("Tuned  Accuracy: {}".format(grid.best_score_))
#    y_pred = grid.predict(X_test)
#    print("Accuracy (usando y_test): {}".format(grid.score(X_test, Y_test)))
#    print(time() - start)

##### El mejor valor fué de 0.72, que como se verá después, equivale a 29 componentes solamente. Un error común es suponer que escoger un elevado número de componentes (o de varianza explicada) equivale a un mejor modelo. Esto es errado y sólo conduce a aumentar el riesgo de sobreajuste. El objetico del PCA es quedarse con lo más rico del modelo sin contaminar el nuevo espacio con ruido. Sólo con 29 componentes principales obtenido mediante transformaciones lineales de un espacio inicial de 785 vectores, se puede lograr un modelo bastante preciso.

##### A continuación se hará el ajuste hiperparamétrico de SVC:

In [12]:
#Tuning SVC (Valores optimos: C=3, gamma=0.04)
#if __name__ == '__main__':
#    svc = SVC()
#    pca =PCA(0.72,whiten=False)
#    steps = [('PCA', pca),('SVC', svc)]
#    pipeline = Pipeline(steps)
#    param_grid = { "SVC__C" : [1,2,3,4,5,6,7,8,9,10] , "SVC__gamma" : [0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,0.1]}
#    grid = GridSearchCV(pipeline, param_grid,cv=3,scoring = "accuracy",iid=False, n_jobs=-1)
#    start = time()
#    grid.fit(X_train, Y_train)
#    print("Tuned Model Parameters: {}".format(grid.best_params_))
#    print("Tuned  Accuracy: {}".format(grid.best_score_))
#    y_pred = grid.predict(X_test)
#    print("Accuracy (usando y_test): {}".format(grid.score(X_test, Y_test)))
#    print(time() - start)

##### Se obtuvo un valor de 3 para la penalización y 0.04 para gamma, que es el coeficiente para el kernel rfb, el cual esta por defecto seleccionado.

##### Los rangos a probar (param_grid) tanto para PCA y SVC inicialmente se realizaron con valores espaciados y luego se fué acotando y aumentando la precisión en la medida que se observó convergencia a un mínimo global de parte de la función de pérdida.

##### if __name__ == '__main__':  seguido del código identado permite setear n_jobs=-1 en ambiente Windows con el objetivo de paralelizar el trabajo en todos los núcleos del computador. En Linux no es necesario realizar aquello y sólo basta con especificar el número de trabajos.

##### Con estos valores hiperparamétricos ya se puede ajustar con la totalidad del set de entrenamiento (42.000 observaciones) y finalmente predecir sobre el  set de test:

In [14]:
X_test_final = test_df.values/255.0

#Tuned Model Parameters: {'SVC__C': 3, 'SVC__gamma': 0.04} PCA(0.72,whiten=False)  Score Kaggle: 0.98200
pca =PCA(0.72,whiten=False)
svc = SVC(C=3, gamma=0.04)
steps = [('PCA', pca),('SVC', svc)]
pipeline_final = Pipeline(steps)
pipeline_final.fit(X, y)
y_pred_final = pipeline_final.predict(X_test_final)
my_solution = pd.DataFrame(y_pred_final)
my_solution.index+=1
my_solution.index.name='ImageId'
my_solution.columns=['Label']
print(my_solution.shape)
my_solution.to_csv("my_solution_SVM_PCA.csv",header=True)


print("Numero de componentes:")
print(pca.n_components_)
print("Varianza explicada:")
print(pca.explained_variance_ratio_.sum())

(28000, 1)
Numero de componentes:
29
Varianza explicada:
0.7249943447516655


## Comentarios Finales

##### El desarrollo anterior pretende ser una guía para personas que se estén adentrando en el mundo de machine learning y ver las metodologías que se pueden implementar para lograr un buen partido con modelos convencionales. Para mejores scores, ya se debería pensar en deep learning.

##### Para realizar el escalado fué suficiente con dividir los vectores predictivos por 255, que es el mayor valor posible para un pixel. Esta estrategia resultó ser la más adecuada en virtud de pruebas anteriores debido a que se mantiene la distribución de los datos en un rango más acotado [0,1].

##### No es necesario utilizar todas las observaciones para hacer un ajuste de hiperparámetros exitoso si la naturaleza de las observaciones es similar (cada observacion está compuesta por números escritos a mano del 0 al 9 llevados a pixeles). Con esta estratégia se gana en tiempo y se obtiene una solución bastante buena.

##### Solo con 29 componentes se obtuvo u una puntuación de 0.98200 en Kaggle, el cual es bastante bueno para un modelo tal como SVC. 

##### Agradeceré toda proposición de mejora de parte de los lectores.